In [344]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [345]:
df = pd.read_csv('student-por.csv')

http://www3.dsi.uminho.pt/pcortez/student.pdf
https://archive.ics.uci.edu/ml/datasets/student%2Bperformance#

In [346]:
print('Rows and Columns :', df.shape,'\n' )

print('INFO:')
print(df.info(), '\n')
print('df Columns:', df.columns, '\n')
print('Unique Values :','\n', df.nunique(), '\n')

Rows and Columns : (649, 33) 

INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 33 columns):
school        649 non-null object
sex           649 non-null object
age           649 non-null int64
address       649 non-null object
famsize       649 non-null object
Pstatus       649 non-null object
Medu          649 non-null int64
Fedu          649 non-null int64
Mjob          649 non-null object
Fjob          649 non-null object
reason        649 non-null object
guardian      649 non-null object
traveltime    649 non-null int64
studytime     649 non-null int64
failures      649 non-null int64
schoolsup     649 non-null object
famsup        649 non-null object
paid          649 non-null object
activities    649 non-null object
nursery       649 non-null object
higher        649 non-null object
internet      649 non-null object
romantic      649 non-null object
famrel        649 non-null int64
freetime      649 non-null int64
goout         64

In [347]:
pd.set_option("display.max_columns", None)
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


In [348]:
df['Mjob'].value_counts()

other       258
services    136
at_home     135
teacher      72
health       48
Name: Mjob, dtype: int64

In [349]:
df['Fjob'].value_counts()

other       367
services    181
at_home      42
teacher      36
health       23
Name: Fjob, dtype: int64

In [350]:
df['failures'].value_counts()

0    549
1     70
2     16
3     14
Name: failures, dtype: int64

dropping extraneous columns

In [351]:
df.drop(['school', 'address', 'Mjob', 'Fjob', 'reason', 'guardian', 'romantic','traveltime', 'nursery','health'], axis = 1, inplace = True)

removing students outside of high school age

In [352]:

df = df[df['age'] <= 18] 
np.sqrt(df.shape[0])

24.657656011875904

In [353]:
df.head(10)

,sex,age,famsize,Pstatus,Medu,Fedu,studytime,failures,schoolsup,famsup,paid,activities,higher,internet,famrel,freetime,goout,Dalc,Walc,absences,G1,G2,G3
0,F,18,GT3,A,4,4,2,0,yes,no,no,no,yes,no,4,3,4,1,1,4,0,11,11
1,F,17,GT3,T,1,1,2,0,no,yes,no,no,yes,yes,5,3,3,1,1,2,9,11,11
2,F,15,LE3,T,1,1,2,0,yes,no,no,no,yes,yes,4,3,2,2,3,6,12,13,12
3,F,15,GT3,T,4,2,3,0,no,yes,no,yes,yes,yes,3,2,2,1,1,0,14,14,14
4,F,16,GT3,T,3,3,2,0,no,yes,no,no,yes,no,4,3,2,1,2,0,11,13,13
5,M,16,LE3,T,4,3,2,0,no,yes,no,yes,yes,yes,5,4,2,1,2,6,12,12,13
6,M,16,LE3,T,2,2,2,0,no,no,no,no,yes,yes,4,4,4,1,1,0,13,12,13
7,F,17,GT3,A,4,4,2,0,yes,yes,no,no,yes,no,4,1,4,1,1,2,10,13,13
8,M,15,LE3,A,3,2,2,0,no,yes,no,no,yes,yes,4,2,2,1,1,0,15,16,17
9,M,15,GT3,T,3,4,2,0,no,yes,no,yes,yes,yes,5,5,1,1,1,0,12,12,13


renaming famsize to siblings and changing value to a binary

In [354]:
df.rename(columns ={'famsize':'siblings'}, inplace = True)

df.loc[(df.siblings == 'GT3'), 'siblings'] = 1
df.loc[(df.siblings == 'LE3'), 'siblings'] = 0

df.head()


,sex,age,siblings,Pstatus,Medu,Fedu,studytime,failures,schoolsup,famsup,paid,activities,higher,internet,famrel,freetime,goout,Dalc,Walc,absences,G1,G2,G3
0,F,18,1,A,4,4,2,0,yes,no,no,no,yes,no,4,3,4,1,1,4,0,11,11
1,F,17,1,T,1,1,2,0,no,yes,no,no,yes,yes,5,3,3,1,1,2,9,11,11
2,F,15,0,T,1,1,2,0,yes,no,no,no,yes,yes,4,3,2,2,3,6,12,13,12
3,F,15,1,T,4,2,3,0,no,yes,no,yes,yes,yes,3,2,2,1,1,0,14,14,14
4,F,16,1,T,3,3,2,0,no,yes,no,no,yes,no,4,3,2,1,2,0,11,13,13


Converting all binary non-numeric variables to numeric variables

In [355]:


df.loc[df.sex== 'F', 'sex'] = 1
df.loc[df.sex == 'M', 'sex'] = 0

df.loc[(df.Pstatus == 'T'), 'Pstatus'] = 2
df.loc[(df.Pstatus == 'A'), 'Pstatus'] = 1

df.loc[(df.schoolsup == 'yes'), 'schoolsup'] = 1
df.loc[(df.schoolsup == 'no'), 'schoolsup'] = 0

df.loc[(df.famsup == 'yes'), 'famsup'] = 1
df.loc[(df.famsup == 'no'), 'famsup'] = 0

df.loc[(df.paid == 'yes'), 'paid'] = 1
df.loc[(df.paid == 'no'), 'paid'] = 0

df.loc[(df.activities == 'yes'), 'activities'] = 1
df.loc[(df.activities == 'no'), 'activities'] = 0

df.loc[(df.higher == 'yes'), 'higher'] = 1
df.loc[(df.higher == 'no'), 'higher'] = 0

df.loc[(df.internet == 'yes'), 'internet'] = 1
df.loc[(df.internet == 'no'), 'internet'] = 0



dummy variables for age column and drop 'age'

In [356]:
ages_dummies = pd.get_dummies(df.age, prefix = 'age').iloc[:,1:]

df = pd.concat([df, ages_dummies], axis = 1)
df.drop('age', axis = 1)


,sex,siblings,Pstatus,Medu,Fedu,studytime,failures,schoolsup,famsup,paid,activities,higher,internet,famrel,freetime,goout,Dalc,Walc,absences,G1,G2,G3,age_16,age_17,age_18
0,1,1,1,4,4,2,0,1,0,0,0,1,0,4,3,4,1,1,4,0,11,11,0,0,1
1,1,1,2,1,1,2,0,0,1,0,0,1,1,5,3,3,1,1,2,9,11,11,0,1,0
2,1,0,2,1,1,2,0,1,0,0,0,1,1,4,3,2,2,3,6,12,13,12,0,0,0
3,1,1,2,4,2,3,0,0,1,0,1,1,1,3,2,2,1,1,0,14,14,14,0,0,0
4,1,1,2,3,3,2,0,0,1,0,0,1,0,4,3,2,1,2,0,11,13,13,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,1,1,2,4,4,1,0,0,1,0,1,1,1,4,4,3,2,2,4,7,9,10,0,0,1
645,1,0,2,3,1,2,0,0,1,0,0,1,1,4,3,4,1,1,4,15,15,16,0,0,1
646,1,1,2,1,1,2,0,0,0,0,1,1,0,1,1,1,1,1,6,11,12,9,0,0,1
647,0,0,2,3,1,1,0,0,0,0,0,1,1,2,4,5,3,4,6,10,10,10,0,1,0


creating target variable quantifying alchohol consumption, and dropping 'Walc' and 'Dalc'

In [357]:
df['week_alc'] = df.Dalc + df.Walc
df.drop(['Walc', 'Dalc'], axis = 1, inplace = True)

feature quantifying famility stability feature

In [358]:
#df['stability'] = (df.Pstatus+df.Fedu + df.Medu)*df.famrel

df['stability'] = df.famrel*df.siblings

feature quantifying academic support network

In [359]:
df['academic_support'] = df.famsup + df.schoolsup  + df.higher 

In [360]:
df['idle'] = df.goout*df.freetime

In [361]:
df['grade_avg'] = round((df.G1 + df.G2 + df.G3)/3, 2)

In [362]:
df.corr()

,sex,age,siblings,Pstatus,Medu,Fedu,studytime,failures,schoolsup,famsup,paid,activities,higher,internet,famrel,freetime,goout,absences,G1,G2,G3,age_16,age_17,age_18,week_alc,stability,academic_support,idle,grade_avg
sex,1.000000,0.061678,0.106622,-0.064509,-0.118618,-0.103302,0.195010,-0.066579,0.107837,0.133660,-0.095795,-0.129585,0.074632,-0.074167,-0.053560,-0.139760,-0.071164,-0.015747,0.109295,0.119147,0.139106,0.003597,0.053626,0.010610,-0.355772,0.077095,0.173040,-0.121921,0.128732
age,0.061678,1.000000,-0.031047,0.001134,-0.066758,-0.063362,0.012776,0.168909,-0.176874,-0.082280,-0.014053,-0.055579,-0.173019,0.022247,0.005798,-0.043773,0.134286,0.117058,-0.073963,-0.030514,-0.021698,-0.352925,0.267574,0.754338,0.106528,-0.009423,-0.208221,0.064999,-0.042588
siblings,0.106622,-0.031047,1.000000,0.225633,0.023704,0.055417,0.023702,0.058023,0.057242,0.044937,0.062859,0.013476,0.012191,-0.003133,-0.004954,0.011083,-0.005222,-0.007985,-0.036791,-0.023153,-0.029123,0.012340,0.048475,-0.064899,-0.101154,0.920420,0.062362,0.010503,-0.030925
Pstatus,-0.064509,0.001134,0.225633,1.000000,-0.065030,-0.035306,-0.021950,0.015951,-0.015639,0.019535,0.010590,0.102127,0.009797,0.047324,0.042436,0.048540,0.031694,-0.135602,0.016018,0.010919,-0.009052,0.039231,-0.002360,-0.011477,0.059394,0.238807,0.010689,0.043259,0.005514
Medu,-0.118618,-0.066758,0.023704,-0.065030,1.000000,0.634022,0.099533,-0.144325,-0.028288,0.120493,0.108464,0.107334,0.180481,0.268041,0.018082,-0.007748,0.006131,0.005004,0.231998,0.243323,0.218090,-0.018126,-0.031149,-0.025775,-0.011656,0.021886,0.145697,-0.009288,0.240945
Fedu,-0.103302,-0.063362,0.055417,-0.035306,0.634022,1.000000,0.039781,-0.125208,0.026179,0.136261,0.090122,0.067301,0.155137,0.181602,0.025196,0.022502,0.024811,0.050811,0.181199,0.200107,0.183927,0.014707,-0.060108,-0.013897,0.033119,0.035391,0.170968,0.018512,0.196762
studytime,0.195010,0.012776,0.023702,-0.021950,0.099533,0.039781,1.000000,-0.138208,0.096260,0.153214,-0.013975,0.045795,0.187303,0.029391,-0.000266,-0.065267,-0.066053,-0.123211,0.255664,0.237789,0.242976,-0.052025,-0.029592,0.050552,-0.201259,0.014531,0.227603,-0.080750,0.256090
failures,-0.066579,0.168909,0.058023,0.015951,-0.144325,-0.125208,-0.138208,1.000000,0.017297,0.035731,0.087780,-0.011344,-0.244602,-0.090057,-0.078254,0.098167,0.070240,0.124750,-0.347481,-0.365465,-0.367176,-0.064235,0.057849,0.119945,0.082391,0.041853,-0.067023,0.101769,-0.376474
schoolsup,0.107837,-0.176874,0.057242,-0.015639,-0.028288,0.026179,0.096260,0.017297,1.000000,0.077077,0.043869,-0.023694,0.089093,-0.035197,-0.018774,-0.009306,-0.049329,-0.053461,-0.084175,-0.065655,-0.079813,0.044066,-0.097803,-0.090390,-0.080892,0.039267,0.541135,-0.029703,-0.079965
famsup,0.133660,-0.082280,0.044937,0.019535,0.120493,0.136261,0.153214,0.035731,0.077077,1.000000,0.100038,-0.001082,0.094462,0.062856,0.033023,0.016053,0.005180,0.046925,0.022322,0.037661,0.050435,-0.005607,0.007496,-0.070905,-0.054980,0.048012,0.776310,-0.003303,0.039213


feature averaging grade

In [363]:
df[df.week_alc > 5].shape

(108, 29)

defining high vs low alcohol consumption

In [364]:
bins = [1, 5, 10]
drinking_categories = [0, 1]

In [365]:
drinking_bins = pd.cut(df.week_alc, bins, labels = drinking_categories)

In [366]:
drinking_bins.rename('light/heavy', inplace = True)

0      0
1      0
2      0
3      0
4      0
      ..
643    0
645    0
646    0
647    1
648    1
Name: light/heavy, Length: 608, dtype: category
Categories (2, int64): [0 < 1]

In [367]:
df = pd.concat([df, drinking_bins], axis = 1)
df.head(20)

,sex,age,siblings,Pstatus,Medu,Fedu,studytime,failures,schoolsup,famsup,paid,activities,higher,internet,famrel,freetime,goout,absences,G1,G2,G3,age_16,age_17,age_18,week_alc,stability,academic_support,idle,grade_avg,light/heavy
0,1,18,1,1,4,4,2,0,1,0,0,0,1,0,4,3,4,4,0,11,11,0,0,1,2,4,2,12,7.33,0
1,1,17,1,2,1,1,2,0,0,1,0,0,1,1,5,3,3,2,9,11,11,0,1,0,2,5,2,9,10.33,0
2,1,15,0,2,1,1,2,0,1,0,0,0,1,1,4,3,2,6,12,13,12,0,0,0,5,0,2,6,12.33,0
3,1,15,1,2,4,2,3,0,0,1,0,1,1,1,3,2,2,0,14,14,14,0,0,0,2,3,2,4,14.00,0
4,1,16,1,2,3,3,2,0,0,1,0,0,1,0,4,3,2,0,11,13,13,1,0,0,3,4,2,6,12.33,0
5,0,16,0,2,4,3,2,0,0,1,0,1,1,1,5,4,2,6,12,12,13,1,0,0,3,0,2,8,12.33,0
6,0,16,0,2,2,2,2,0,0,0,0,0,1,1,4,4,4,0,13,12,13,1,0,0,2,0,1,16,12.67,0
7,1,17,1,1,4,4,2,0,1,1,0,0,1,0,4,1,4,2,10,13,13,0,1,0,2,4,3,4,12.00,0
8,0,15,0,1,3,2,2,0,0,1,0,0,1,1,4,2,2,0,15,16,17,0,0,0,2,0,2,4,16.00,0
9,0,15,1,2,3,4,2,0,0,1,0,1,1,1,5,5,1,0,12,12,13,0,0,0,2,5,2,5,12.33,0


initializing x and target 

In [368]:

X = df[['stability','academic_support','idle','grade_avg']]

y = df['light/heavy']

Splitting data for training and testing

In [369]:
sac_train, sac_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 13)

In [370]:
sac_train


,stability,academic_support,idle,grade_avg
148,5,1,25,8.67
145,4,2,6,9.33
263,5,0,20,7.67
197,0,2,9,13.67
52,0,1,25,9.33
...,...,...,...,...
154,0,1,16,10.67
558,5,0,25,10.33
74,4,3,9,11.00
178,0,0,15,8.00


upsampling the minority class in test data

In [371]:
from imblearn.over_sampling import SMOTE

In [372]:
smt = SMOTE(random_state = 13)
sac_smote, y_smote = smt.fit_sample(sac_train, y_train)

fitting early logistic regression model

In [373]:
logreg = LogisticRegression(solver = 'liblinear')

In [374]:
log = logreg.fit(sac_smote, y_smote)

evaluating model performance

In [375]:
y_pred = logreg.predict(sac_test)

In [376]:
confusion_matrix(y_test, y_pred)

array([[74, 31],
       [ 8,  9]])

In [377]:
f1_score(y_test, y_pred)

0.31578947368421056

In [10]:
conditions=[df['Mjob'] == 'at_home',
          df['Mjob'] == 'services',
          df['Mjob'] == 'teacher',
          df['Mjob'] == 'health',
           df['Mjob']== 'other']
choices = [1,2,3,4,5]
df['Mjob'] = np.select(conditions, choices)

conditions=[df['Fjob'] == 'at_home',
          df['Fjob'] == 'services',
          df['Fjob'] == 'teacher',
          df['Fjob'] == 'health',
           df['Fjob']== 'other']
choices = [1,2,3,4,5]
df['Fjob'] = np.select(conditions, choices)

conditions=[df['reason'] == 'home',
          df['reason'] == 'reputation',
          df['reason'] == 'course',
           df['reason']== 'other']
choices = [1,2,3,4]
df['reason'] = np.select(conditions, choices)

conditions=[df['guardian'] == 'father',
          df['guardian'] == 'mother',
          df['guardian'] == 'other']
choices = [1,2,3]
df['guardian'] = np.select(conditions, choices)


In [11]:
df['Mjob'].value_counts()

5    258
2    136
1    135
3     72
4     48
Name: Mjob, dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 34 columns):
school        649 non-null object
sex           649 non-null object
age           649 non-null int64
address       649 non-null object
famsize       649 non-null object
Pstatus       649 non-null object
Medu          649 non-null int64
Fedu          649 non-null int64
Mjob          649 non-null int64
Fjob          649 non-null int64
reason        649 non-null int64
guardian      649 non-null int64
traveltime    649 non-null int64
studytime     649 non-null int64
failures      649 non-null int64
schoolsup     649 non-null object
famsup        649 non-null object
paid          649 non-null object
activities    649 non-null object
nursery       649 non-null object
higher        649 non-null object
internet      649 non-null object
romantic      649 non-null object
famrel        649 non-null int64
freetime      649 non-null int64
goout         649 non-null int64
Dalc          649 non-nu

In [15]:
df['alc']= df['Dalc']+df['Walc']

df['alc'].value_counts()

2     241
3     116
4      99
5      73
6      50
7      32
8      17
10     15
9       6
Name: alc, dtype: int64

need to answer why using this type of binning? How does it help answer our business problem.

In [17]:
# if >=6 then heavy drinker, 1
df['alc'].where(df['alc']>=6, 0, inplace = True)
df['alc'].where(df['alc']<=5, 1, inplace = True)

print(df['alc'].value_counts())
print(df['alc'].value_counts(normalize = True))


0    529
1    120
Name: alc, dtype: int64
0    0.8151
1    0.1849
Name: alc, dtype: float64


In [24]:
object_cols= df.select_dtypes(include=object).columns

In [25]:
# converting object columns to numeric 
for col in object_cols:
    df[col]= pd.to_numeric(df[col])
    
df.dtypes

school        int64
sex           int64
age           int64
address       int64
famsize       int64
Pstatus       int64
Medu          int64
Fedu          int64
Mjob          int64
Fjob          int64
reason        int64
guardian      int64
traveltime    int64
studytime     int64
failures      int64
schoolsup     int64
famsup        int64
paid          int64
activities    int64
nursery       int64
higher        int64
internet      int64
romantic      int64
famrel        int64
freetime      int64
goout         int64
Dalc          int64
Walc          int64
health        int64
absences      int64
G1            int64
G2            int64
G3            int64
alc           int64
dtype: object

In [26]:
df.isna().any()

school        False
sex           False
age           False
address       False
famsize       False
Pstatus       False
Medu          False
Fedu          False
Mjob          False
Fjob          False
reason        False
guardian      False
traveltime    False
studytime     False
failures      False
schoolsup     False
famsup        False
paid          False
activities    False
nursery       False
higher        False
internet      False
romantic      False
famrel        False
freetime      False
goout         False
Dalc          False
Walc          False
health        False
absences      False
G1            False
G2            False
G3            False
alc           False
dtype: bool

**Note:** How can you use this info during the modeling phase? Can you perform any variable significance to check this, for values only upto 18 years, or also more?

In [31]:
df[df['age'] >=19]['age'].count()

41